In [5]:
#!pip install kaggle 
#!pip install missingno
#!pip install scikit-image

   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.9 MB 16.8 MB/s eta 0:00:01
   --------------------- ------------------ 6.8/12.9 MB 19.6 MB/s eta 0:00:01
   ------------------------------------ --- 11.8/12.9 MB 20.2 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 20.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 45.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import requests
import pandas as pd
import os

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import missingno as msno
from datetime import date

from skimage.feature import shape_index
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)


from kaggle.api.kaggle_api_extended import KaggleApi

In [2]:
#api = KaggleApi()
#api.authenticate()

#api.dataset_download_files('olistbr/brazilian-ecommerce', path='./dataset', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce


In [8]:
# Bu dosyanın (script) bulunduğu klasörü baz alıyoruz
base_path = os.path.join(os.getcwd(), "dataset")

file_name_customers = "olist_customers_dataset.csv"
file_name_geolocation = "olist_geolocation_dataset.csv"
file_name_orderitems = "olist_order_items_dataset.csv"
file_name_orderpayments = "olist_order_payments_dataset.csv"
file_name_orderreviews = "olist_order_reviews_dataset.csv"
file_name_orders = "olist_orders_dataset.csv"
file_name_products = "olist_products_dataset.csv"
file_name_sellers = "olist_sellers_dataset.csv"
file_name_category_name_translation = "product_category_name_translation.csv"


# Tam yol
customers_path = os.path.join(base_path, file_name_customers)
geolocation_path = os.path.join(base_path, file_name_geolocation)
orderitems_path = os.path.join(base_path, file_name_orderitems)
orderpayments_path = os.path.join(base_path, file_name_orderpayments)
orderreviews_path = os.path.join(base_path, file_name_orderreviews)
orders_path = os.path.join(base_path, file_name_orders)
products_path = os.path.join(base_path, file_name_products)
sellers_path = os.path.join(base_path, file_name_sellers)
product_category_name_translation_path = os.path.join(base_path, file_name_category_name_translation)

def load(dataset):
    try:
        data = pd.read_csv(dataset)
        return data
    except FileNotFoundError:
        print(f"Dosya bulunamadı: {dataset}")
        return None

In [9]:
df_customers = load(customers_path)
df_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [10]:
df_geolocation = load(geolocation_path)
df_geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.546,-46.639,sao paulo,SP
1,1046,-23.546,-46.645,sao paulo,SP
2,1046,-23.546,-46.643,sao paulo,SP
3,1041,-23.544,-46.639,sao paulo,SP
4,1035,-23.542,-46.642,sao paulo,SP


In [11]:
df_orderitems = load(orderitems_path)
df_orderitems.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.900,13.290
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.900,19.930
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.000,17.870
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.990,12.790
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.900,18.140


In [12]:
df_orderpayments = load(orderpayments_path)
df_orderpayments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.330
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.390
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.710
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.780
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.450


In [13]:
df_orderreviews = load(orderreviews_path)
df_orderreviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [14]:
df_orders = load(orders_path)
df_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [15]:
df_products = load(products_path)
df_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.000,287.000,1.000,225.000,16.000,10.000,14.000
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.000,276.000,1.000,1000.000,30.000,18.000,20.000
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.000,250.000,1.000,154.000,18.000,9.000,15.000
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.000,261.000,1.000,371.000,26.000,4.000,26.000
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.000,402.000,4.000,625.000,20.000,17.000,13.000


In [16]:
df_sellers = load(sellers_path)
df_sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


# DATA MODELING

-----------------------------
️1-  Basit sayısal ID oluşturma fonksiyonu
-----------------------------

In [17]:
def create_simple_id(df, original_id_col):
    """Var olan ID sütunundan sayısal ID oluşturur"""
    df = df.copy()
    df['simple_id'] = pd.factorize(df[original_id_col])[0] + 1
    return df

def create_simple_id_no_col(df):
    """ID sütunu olmayan tablolar için sayısal ID oluşturur"""
    df = df.copy()
    df['simple_id'] = range(1, len(df) + 1)
    return df

def create_prefixed_id(df, new_id_col, prefix):
    """Sayısal ID'ye prefiks ekler"""
    df = df.copy()
    df[new_id_col] = [f"{prefix}{str(i).zfill(3)}" for i in df['simple_id']]
    return df

️2- Tüm tablolar için basit ID oluşturma
-----------------------------


In [20]:
# Var olan ID sütunları
df_customers = create_simple_id(df_customers, 'customer_id')
df_sellers = create_simple_id(df_sellers, 'seller_id')
df_orders = create_simple_id(df_orders, 'order_id')
df_orderitems = create_simple_id(df_orderitems, 'order_item_id')
df_products = create_simple_id(df_products, 'product_id')

# ID olmayan tablolar
df_orderpayments = create_simple_id_no_col(df_orderpayments)
df_geolocation = create_simple_id_no_col(df_geolocation)


️3- Prefiksli ID ekleme
-----------------------------

In [21]:
df_customers = create_prefixed_id(df_customers, 'customer_simple_id', 'CUS')
df_sellers = create_prefixed_id(df_sellers, 'seller_simple_id', 'SEL')
df_orders = create_prefixed_id(df_orders, 'order_simple_id', 'ORD')
df_orderitems = create_prefixed_id(df_orderitems, 'order_item_simple_id', 'OIT')
df_products = create_prefixed_id(df_products, 'product_simple_id', 'PRO')
df_orderpayments = create_prefixed_id(df_orderpayments, 'orderpayment_simple_id', 'PAY')
df_geolocation = create_prefixed_id(df_geolocation, 'geolocation_simple_id', 'GEO')


Zip code prefix sütunlarını standardize etme
-----------------------------

In [22]:
df_customers.rename(columns={'customer_zip_code_prefix':'zip_code_prefix'}, inplace=True)
df_sellers.rename(columns={'seller_zip_code_prefix':'zip_code_prefix'}, inplace=True)
if 'customer_zip_code_prefix' in df_orders.columns:
    df_orders.rename(columns={'customer_zip_code_prefix':'zip_code_prefix'}, inplace=True)

# df_geolocation tablosu için
df_geolocation.rename(columns={'geolocation_zip_code_prefix':'zip_code_prefix'}, inplace=True)


Bu kodlama ile:

Tüm tablolar için basit, tekil ID’ler oluşturuldu.

df_geolocation tablosuna da bir ID eklendi.

Zip code prefix sütunları tüm tablolar için standart hale getirildi.

## Eski uzun ID lerin Silinmesi ve Tablolarin Duzenlenmesi

In [31]:
# Orijinal ID ve simple_id sütunlarını listeliyoruz
original_id_cols = {
    'df_customers': ['customer_id', 'customer_unique_id', 'simple_id'],
    'df_sellers': ['seller_id', 'unique_id', 'simple_id'],
    'df_orders': ['order_id', 'customer_id', 'unique_id', 'simple_id'],
    'df_orderitems': ['order_item_id', 'order_id', 'product_id', 'seller_id', 'unique_id', 'simple_id'],
    'df_products': ['product_id', 'unique_id', 'simple_id'],
    'df_orderpayments': ['unique_id', 'simple_id'],  # ID yok, sadece unique_id ve simple_id varsa
    'df_geolocation': ['geolocation_zip_code_prefix', 'unique_id', 'simple_id']  # zip code hariç tutulabilir
}

# Tabloları sözlükte tutuyoruz
dfs = {
    'df_customers': df_customers,
    'df_sellers': df_sellers,
    'df_orders': df_orders,
    'df_orderitems': df_orderitems,
    'df_products': df_products,
    'df_orderpayments': df_orderpayments,
    'df_geolocation': df_geolocation
}


In [32]:
# Prefiksli ID'yi ilk sutuna tasima:

def move_prefixed_id_first(df, prefixed_id_col):
    df = df.copy()
    cols = [prefixed_id_col] + [c for c in df.columns if c != prefixed_id_col]
    return df[cols]

# Prefiksli ID sütunları
prefixed_cols = {
    'df_customers': 'customer_simple_id',
    'df_sellers': 'seller_simple_id',
    'df_orders': 'order_simple_id',
    'df_orderitems': 'order_item_simple_id',
    'df_products': 'product_simple_id',
    'df_orderpayments': 'orderpayment_simple_id',
    'df_geolocation': 'geolocation_simple_id'
}

In [33]:
#  orijinal ve unique_id sütunlarının silinmesi ve   prefiksli ID'lerin ilk sutun olarak belirlenmesi
# İşlem: önce orijinal ve eski simple_id sütunlarını sil, sonra prefiksli ID'yi başa taşı
for key, df in dfs.items():
    df.drop(columns=[c for c in original_id_cols[key] if c in df.columns and c != prefixed_cols[key]], inplace=True)
    dfs[key] = move_prefixed_id_first(df, prefixed_cols[key])


In [34]:
# Tabloları tekrar değişkenlere atıyoruz

(df_customers, df_sellers, df_orders, df_orderitems,
 df_products, df_orderpayments, df_geolocation) = [dfs[k] for k in dfs]

## Olusturulan yeni tablolarin disariya aktarilmasi
(kok dizin icine aktarilir)

### Data tablolari yeni klasore (cleaned_data_with_prefix_id) eklenmis durumda. Mevcut yapiyi dosyaniza almak icin sadece pull ediniz. 
### Kodu yeniden calistirirsaniz ikinci bir data klasoru olusturabilirsiniz.

In [35]:

# ❗ Not: Bu kod her çalıştırıldığında yeni CSV dosyaları oluşturur.
# Eğer mevcut temizlenmiş datasetlerin üzerine yazmak istemiyorsanız çalıştırmayın.
# Bu kodun amacı: Prefiksli ID'lerle güncellenmiş tüm tabloları CSV olarak dışarı aktarmaktır.

proceed = input("CSV dosyalarını yeniden oluşturmak istediğinize emin misiniz? (E/H): ")

if proceed.upper() != 'E':
    print("İşlem iptal edildi.")
else:
    # Kayıt klasörünü tanımla
    base_path = os.path.join(os.getcwd(), "cleaned_data_with_prefix_id")
    os.makedirs(base_path, exist_ok=True)  # Klasör yoksa oluştur

    # Çıktı dosya yolları
    output_files = {
        'df_customers': os.path.join(base_path, "customers_cleaned.csv"),
        'df_sellers': os.path.join(base_path, "sellers_cleaned.csv"),
        'df_orders': os.path.join(base_path, "orders_cleaned.csv"),
        'df_orderitems': os.path.join(base_path, "orderitems_cleaned.csv"),
        'df_products': os.path.join(base_path, "products_cleaned.csv"),
        'df_orderpayments': os.path.join(base_path, "orderpayments_cleaned.csv"),
        'df_geolocation': os.path.join(base_path, "geolocation_cleaned.csv")
    }

    # Her tabloyu CSV olarak kaydet
    for key, filename in output_files.items():
        dfs[key].to_csv(filename, index=False)
        print(f"{filename} kaydedildi ✅")



CSV dosyalarını yeniden oluşturmak istediğinize emin misiniz? (E/H):  E


D:\EuroTech_DataScience\Proje\Git_Proje\cleaned_data_with_prefix_id\customers_cleaned.csv kaydedildi ✅
D:\EuroTech_DataScience\Proje\Git_Proje\cleaned_data_with_prefix_id\sellers_cleaned.csv kaydedildi ✅
D:\EuroTech_DataScience\Proje\Git_Proje\cleaned_data_with_prefix_id\orders_cleaned.csv kaydedildi ✅
D:\EuroTech_DataScience\Proje\Git_Proje\cleaned_data_with_prefix_id\orderitems_cleaned.csv kaydedildi ✅
D:\EuroTech_DataScience\Proje\Git_Proje\cleaned_data_with_prefix_id\products_cleaned.csv kaydedildi ✅
D:\EuroTech_DataScience\Proje\Git_Proje\cleaned_data_with_prefix_id\orderpayments_cleaned.csv kaydedildi ✅
D:\EuroTech_DataScience\Proje\Git_Proje\cleaned_data_with_prefix_id\geolocation_cleaned.csv kaydedildi ✅
